In [ ]:
%matplotlib inline

In [ ]:
import os
import pymysql
import pandas as pd
import numpy as np

import matplotlib as plt

In [ ]:
infilepath = os.path.join('inputs', 'M1_detectors_Luton-to-MK.csv')
detectors_xy = pd.read_csv(infilepath, index_col=0)

In [ ]:
def fetch_norm_sql(sql, conn):
    print(sql, conn)
    #df = pd.read_sql(sql, conn)
    #df = clean_df(df)
    #norm_data = normalise_data(df)
    #df = norm_data['Lane']
    #df_resampled = aggr_intervals(df, interval)
    
    #return df_resampled
    return sql

In [ ]:
import multiprocessing
from itertools import product

if __name__ == '__main__':
    """
    conn = pymysql.connect(host='TSCDSSQL01',
                           user='haris',
                           passwd='LihuvGPW4jgbEQ0AyVta!',
                           db='MAIA')
    """
    nprocs = 3
    sql_detectors = list(detectors_xy.index.values)
    sql_detectors = ['M1/2655A', 'M1/2662A']

    sqls = []
    for det in sql_detectors:
        sql = "SELECT * from midas_Co30_all_2015 WHERE Geographic_Address = '{}'".format(det)
        sqls.append(sql)

    conn = 1

    with multiprocessing.Pool(processes=1) as pool:
        res = pool.starmap(fetch_norm_sql, product(sql_detectors, repeat=2))

    #conn.close()

    #midas = pd.concat(dfs)


In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 50)

In [ ]:
def clean_df(df):
    # Replace -1 with NaN, will be used to drop NaN columns
    df.replace(-1, np.nan, inplace=True)
    df.dropna(1, 'all', inplace=True)
    cols_to_drop = ['CO_Address', 'LCC_Address', 'Transponder_Address', 'Device_Address', 'Number_of_Lanes']
    df.drop(cols_to_drop, axis=1, inplace=True)
    df.rename(columns={'Geographic_Address': 'GA'}, inplace=True)
    return df

In [ ]:
def normalise_data(df):
    # Create a dictionary with the variable combinations
    categories = dict()

    for c in df:
        if '_' in c:
            var, cat, num = c.split('_', 2)
            key = '{}-{}'.format(cat, var)

            if key in categories:
                categories[key].append(num)
            else:
                categories[key] = list(num)

    d_results = dict()

    id_vars = ['GA', 'Datetime']
    for k in categories:
        cat, var = k.split('-')
        for c in df:
            if (cat in c) and (var in c):
                value_vars = list()
                for n in categories[k]:
                    col_name = '{}_{}_{}'.format(var, cat, n)
                    value_vars.append(col_name)

        cols_tmp = [*id_vars, *value_vars]
        
        dft = df[cols_tmp]
        dft = pd.melt(dft, id_vars=id_vars, value_vars=value_vars , var_name=cat, value_name=var)
        dft[cat].replace(regex=True, to_replace=r'\D', value=r'', inplace=True)
        dft[cat] = dft[cat].astype('int')
        dft.set_index(list(dft.columns[:-1]), inplace=True)
        
        # Dictionary with all the results
        d_results.setdefault(cat, {})[var] = dft
    
    data = {}
    for k1 in d_results:
        dfs = []
        for k2, v in d_results[k1].items():
            dfs.append(v)
        df = pd.concat(dfs, axis=1)
        df = df.swaplevel(0, 1)
        df.sort_index(inplace=True)
        data[k1] = df
    
    return data

In [ ]:
def aggr_intervals(df, interval):
    d_aggrs = {'Speed': 'mean', 'Flow':'sum', 'Occupancy':'mean', 'Headway':'mean'}
    # Due to dropping during cleaning there is a chance a whole column is gone
    # Restore with a NaN column
    for k in d_aggrs:
        if not k in df:
            df.insert(0, k, np.nan)

    df_resamped = df.reset_index(level=[1, 2]).groupby('GA').resample(interval).agg(d_aggrs)
    df_resamped = df_resamped.round(2)
    
    return df_resamped

# Imports

In [ ]:
conn = pymysql.connect(host='TSCDSSQL01',
                       user='haris',
                       passwd='LihuvGPW4jgbEQ0AyVta!',
                       db='MAIA')
cur = conn.cursor()

interval = '15Min'

dfs = []
# Read the db and convert it to df
for det in sql_detectors:
    print('{}/{}'.format(i, len(sql_detectors)))
    sql = "SELECT * from midas_Co30_all_2015 WHERE Geographic_Address = '{}'".format(det)   
    df = pd.read_sql(sql, conn)
    df = clean_df(df)
    norm_data = normalise_data(df)
    df = norm_data['Lane']
    df_resampled = aggr_intervals(df, interval)
    dfs.append(df_resampled)

# Close the connection
cur.close()
conn.close()

midas = pd.concat(dfs)

In [ ]:
conn = pymysql.connect(host='TSCDSSQL01',
                       user='haris',
                       passwd='LihuvGPW4jgbEQ0AyVta!',
                       db='MAIA')
cur = conn.cursor()

interval = '15Min'

dfs = []
# Read the db and convert it to df
i = 1
for det in sql_detectors:
    print('{}/{}'.format(i, len(sql_detectors)))
    sql = "SELECT * from midas_Co30_all_2015 WHERE Geographic_Address = '{}'".format(det)   
    df = pd.read_sql(sql, conn)
    df = clean_df(df)
    norm_data = normalise_data(df)
    df = norm_data['Lane']
    df_resampled = aggr_intervals(df, interval)
    dfs.append(df_resampled)
    i += 1
# Close the connection
cur.close()
conn.close()

midas = pd.concat(dfs)

In [ ]:
dfs

In [ ]:
# Check for columns that contain nans
print('Check for columns with nans\n', midas.isnull().any())

In [ ]:
midas.memory_usage()

In [ ]:
sql_detectors = tuple(detectors_xy.index.values)

In [ ]:
sql

In [ ]:
%%time

conn = pymysql.connect(host='TSCDSSQL01',
                       user='haris',
                       passwd='LihuvGPW4jgbEQ0AyVta!',
                       db='MAIA')
cur = conn.cursor()

interval = '15Min'

# Read the db and convert it to df
sql = "SELECT * from midas_Co30_all_2015 WHERE Geographic_Address IN {}".format(sql_detectors)
df = pd.read_sql(sql, conn)

# Close the connection
cur.close()
conn.close()

df = clean_df(df)
midas = normalise_data(df)

df = midas['Lane']
df_resampled = aggr_intervals(df, '15Min')

# Cleaning

# Analysis

In [ ]:
midas

In [ ]:
midas.memory_usage()

In [ ]:
idxslc = pd.IndexSlice

In [ ]:
df = data['Lane']

In [ ]:
df.head(2)

In [ ]:
df_lanes_avg = df.groupby(level=[0,1]).mean()

In [ ]:
df = df_resamped.copy()

In [ ]:
dg = dict(zip(df.index.get_level_values('Datetime'), df.index.get_level_values('Datetime').time))

In [ ]:
avg_profiles = df.groupby([None, dg], level=[0,1]).mean()

In [ ]:
avg_profiles.describe()

In [ ]:
avg_profiles.plot.scatter(0, 1)

In [ ]:
dft = df_resamped.loc[idxslc['M1/2655A', '2015-04-13'], :]
dft.reset_index(level=0).groupby('GA').resample('10Min').mean()

dg = dict(zip(df.index.get_level_values('Datetime'), df.index.get_level_values('Datetime').time))
dftg = dft.groupby([None, dg], level=[0,1]).mean()
dftg

In [ ]:
dftg.sub(avg_profiles).dropna().Speed.plot()

In [ ]:
def update_plot(data, var, timerange, freq):    
    
    var=list(var)
    df = data[var]
    start_time = timerange[0]
    end_time = timerange[1]
    
    df = df.between_time(timerange[0], timerange[1])

    if freq > 0:
        freq = '{}Min'.format(freq)
        dfg = df.groupby(pd.Grouper(freq=freq)).mean()
        #title = '{} - {}'.format(dfg.name, freq)
        ax = dfg.plot()
        #ax.set_ylabel(data.name)
        return ax
    else:
        print('Frequency cannot be 0')
        

In [ ]:
timerange = detector_data.index.strftime('%X')
descr_time = 'Time:{} - {}'.format(timerange[0], timerange[-1])

lyt=Layout(width='30%', height='30px')
style = {'description_width': 'initial'}

interact(update_plot, data=fixed(detector_data), 
         var=SelectMultiple(options=variables, value=[variables[0]], description='Variable:'),
         timerange=SelectionRangeSlider(options=timerange, value=(timerange[0], timerange[-1]), description=descr_time, style=style, layout=lyt),
         freq=IntSlider(min=0, max=180, step=5, value=5, description='Freq:', layout=lyt),
        )


In [ ]:
widgets.SelectMultiple(
    options=['Apples', 'Oranges', 'Pears'],
    value=['Oranges'],
    #rows=10,
    description='Fruits',
    disabled=False